In [4]:
#import necessary libraries
import pandas as pd


import requests
from bs4 import BeautifulSoup as bs

import warnings
warnings.filterwarnings('ignore')


In [5]:
#create a function to scrape features from the website
def scrape_features(page_number):
    #source url
    source_url = 'https://www.politifact.com/factchecks/list/?page='
    #url to the pages containing the needed features
    page_url = source_url + str(page_number)

    # Send a get request to the page url
    response = requests.get(page_url)

    # Parse the HTML content with BeautifulSoup
    soup = bs(response.text, 'html.parser')

    #create a list to store the scrapped features
    Author = []
    Source = []
    News_statement = []
    Validity_check = []
        
    #author's tag
    m_statement_footer = soup.find_all('footer', class_= 'm-statement__footer')
    #get date and author's name
    for x in m_statement_footer:
        x = x.text.strip()
        x = x.split()
        
        #get author's name
        first_name = x[1]
        last_name = x[2]
        name = first_name + ' ' + last_name
        Author.append(name)
               
    #news statement tag
    m_statement_quote = soup.find_all('div', class_ = 'm-statement__quote')
    #get news_statement
    for x in m_statement_quote:
        x = x.find('a')
        x = x.text.strip()
        news_statement = x
        News_statement.append(news_statement)    
           
    #source tag
    m_statement_meta = soup.find_all('div', class_ = 'm-statement__meta')
    #get source
    for x in m_statement_meta:
        x = x.find('a')
        x = x.text.strip()
        source = x
        Source.append(source)
            
    #news validity tag
    m_statement_meter = soup.find_all('div', class_ = 'm-statement__meter')
    #get news_validity
    for x in m_statement_meter:
        validity = x.find('div', class_ = 'c-image').find('img').get('alt')
        Validity_check.append(validity)
    
    
    #create a dataframe for the scrapped article
    df = pd.DataFrame()
    df['Author'] = Author
    df['News_statement'] = News_statement
    df['Source'] = Source
    df['Validity_check'] = Validity_check
    

    return df

In [8]:
dfs = []
#create a loop for the number of pages
for page in range(1, 501):
    dfs.append(scrape_features(page))

df = pd.concat(dfs, ignore_index=True)


In [9]:
df

,Author,News_statement,Source,Validity_check
0,Andy Nguyen,"The ocean is ""flat"" and ""contained"" by land fe...",Facebook posts,false
1,Jeff Cercone,"""Healthy pilots are suffering from myocarditis...",Instagram posts,false
2,Tom Kertscher,"“Face masks may raise risk of stillbirths, tes...",Instagram posts,barely-true
3,Sara Swann,“When fentanyl burns it smells like popcorn.”,Facebook posts,false
4,Loreben Tuquero,Chick-fil-A “donates another $1.8 million to a...,Instagram posts,false
...,...,...,...,...
14845,Tom Kertscher,All members of Congress are required to be in ...,Sean Duffy,half-true
14846,Caryn Shinske,"""Property taxes have increased 20 percent unde...",Milly Silva,mostly-true
14847,Dave Umhoefer,"""We’re 49th in the United States in new busine...",Mary Burke,mostly-true
14848,C. Eugene,"""Local law enforcement . . . will have access ...",Evan Feinberg,false


In [10]:
# save the DataFrame to a CSV file
df.to_csv('NEWS.csv', index=False)